In [25]:
import pandas as pd
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

In [26]:
data = pd.read_csv("/kaggle/input/restaurant-dataset/Cognifyz_Dataset.csv")
data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [28]:
data.isna().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [29]:
real_data = data.drop(['Country Code','City','Address','Locality','Locality Verbose','Longitude','Latitude','Currency','Average Cost for two','Has Table booking',
                       'Has Online delivery','Is delivering now','Switch to order menu','Rating color','Rating text'],axis = 1)
real_data.head()

,Restaurant ID,Restaurant Name,Cuisines,Price range,Aggregate rating,Votes
0,6317637,Le Petit Souffle,"French, Japanese, Desserts",3,4.8,314
1,6304287,Izakaya Kikufuji,Japanese,3,4.5,591
2,6300002,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4,4.4,270
3,6318506,Ooma,"Japanese, Sushi",4,4.9,365
4,6314302,Sambo Kojin,"Japanese, Korean",4,4.8,229


In [30]:
real_data.isnull().sum()
real_data.dropna(how='any',inplace=True)

In [31]:
real_data['Restaurant Name'].duplicated().sum()

2105

In [32]:
real_data['Restaurant Name'].value_counts()

Restaurant Name
Cafe Coffee Day             83
Domino's Pizza              79
Subway                      63
Green Chick Chop            51
McDonald's                  48
                            ..
The Town House Cafe          1
The G.T. Road                1
The Darzi Bar & Kitchen      1
Smoke On Water               1
Walter's Coffee Roastery     1
Name: count, Length: 7437, dtype: int64

In [33]:
real_data = real_data.sort_values(by = ['Restaurant Name','Aggregate rating'],ascending=False)
real_data.head()

,Restaurant ID,Restaurant Name,Cuisines,Price range,Aggregate rating,Votes
9523,6000871,�ukura��a Sofras۱,"Kebab, Izgara",3,4.4,296
3120,18222559,{Niche} - Cafe & Bar,"North Indian, Chinese, Italian, Continental",3,4.1,492
9334,7100938,wagamama,"Japanese, Asian",4,3.7,131
9454,6401789,tashas,"Cafe, Mediterranean",4,4.1,374
4659,18361747,t Lounge by Dilmah,"Cafe, Tea, Desserts",2,3.6,34


In [34]:
rest_dup = real_data[real_data['Restaurant Name'] == 'Subway']
rest_dup.head()

real_data = real_data.drop_duplicates('Restaurant Name', keep='first')

rest_dup = real_data[real_data['Restaurant Name'] == 'Subway']
rest_dup.head()

,Restaurant ID,Restaurant Name,Cuisines,Price range,Aggregate rating,Votes
3583,306031,Subway,"American, Fast Food, Salad, Healthy Food",2,3.8,97


In [35]:
real_data.shape

(7437, 6)

In [36]:
real_data['Cuisines'] = real_data['Cuisines'].str.split(', ')

In [37]:
real_data

,Restaurant ID,Restaurant Name,Cuisines,Price range,Aggregate rating,Votes
9523,6000871,�ukura��a Sofras۱,"[Kebab, Izgara]",3,4.4,296
3120,18222559,{Niche} - Cafe & Bar,"[North Indian, Chinese, Italian, Continental]",3,4.1,492
9334,7100938,wagamama,"[Japanese, Asian]",4,3.7,131
9454,6401789,tashas,"[Cafe, Mediterranean]",4,4.1,374
4659,18361747,t Lounge by Dilmah,"[Cafe, Tea, Desserts]",2,3.6,34
...,...,...,...,...,...,...
8692,18317511,#Urban Caf��,"[North Indian, Chinese, Italian]",2,3.3,49
6998,18336489,#OFF Campus,"[Cafe, Continental, Italian, Fast Food]",2,3.7,216
2613,18311951,#InstaFreeze,[Ice Cream],1,0.0,2
9148,18378803,#Dilliwaala6,[North Indian],3,3.7,124


In [38]:
real_data = real_data.explode('Cuisines')
real_data

,Restaurant ID,Restaurant Name,Cuisines,Price range,Aggregate rating,Votes
9523,6000871,�ukura��a Sofras۱,Kebab,3,4.4,296
9523,6000871,�ukura��a Sofras۱,Izgara,3,4.4,296
3120,18222559,{Niche} - Cafe & Bar,North Indian,3,4.1,492
3120,18222559,{Niche} - Cafe & Bar,Chinese,3,4.1,492
3120,18222559,{Niche} - Cafe & Bar,Italian,3,4.1,492
...,...,...,...,...,...,...
6998,18336489,#OFF Campus,Italian,2,3.7,216
6998,18336489,#OFF Campus,Fast Food,2,3.7,216
2613,18311951,#InstaFreeze,Ice Cream,1,0.0,2
9148,18378803,#Dilliwaala6,North Indian,3,3.7,124


In [39]:
real_data['Cuisines'].value_counts()

Cuisines
North Indian         3271
Chinese              2290
Fast Food            1262
Mughlai               778
Continental           670
                     ... 
Armenian                1
Gourmet Fast Food       1
Durban                  1
Iranian                 1
Bubble Tea              1
Name: count, Length: 145, dtype: int64

In [40]:
encode = pd.crosstab(real_data['Restaurant Name'],real_data['Cuisines'])

In [41]:
encode

Cuisines,Afghani,African,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Assamese,...,Teriyaki,Tex-Mex,Thai,Tibetan,Turkish,Turkish Pizza,Vegetarian,Vietnamese,Western,World Cuisine
Restaurant Name,,,,,,,,,,,,,,,,,,,,,
#45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#Dilliwaala6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#InstaFreeze,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#OFF Campus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#Urban Caf��,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t Lounge by Dilmah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tashas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wagamama,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
jac_dist = pdist(encode.values, metric='jaccard')
jac_mat = squareform(jac_dist)

jac_sim = 1 - jac_mat
df_jac = pd.DataFrame(jac_sim,index=encode.index,columns=encode.index)

In [43]:
df_jac

Restaurant Name,#45,#Dilliwaala6,#InstaFreeze,#OFF Campus,#Urban Caf��,#hashtag,'Ohana,10 Downing Street,10 To 10 In Delhi,11th Avenue Cafe Bistro,...,feel ALIVE,hug!,iGNiTE,iKitchen,sketch Gallery,t Lounge by Dilmah,tashas,wagamama,{Niche} - Cafe & Bar,�ukura��a Sofras۱
Restaurant Name,,,,,,,,,,,,,,,,,,,,,
#45,1.000000,0.000000,0.0,0.250000,0.000000,1.000000,0.0,0.000000,0.500000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.50,0.0,0.000000,0.0
#Dilliwaala6,0.000000,1.000000,0.0,0.000000,0.333333,0.000000,0.0,0.500000,0.000000,0.000000,...,0.250000,0.000000,0.500000,0.333333,0.0,0.000000,0.00,0.0,0.250000,0.0
#InstaFreeze,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0
#OFF Campus,0.250000,0.000000,0.0,1.000000,0.166667,0.250000,0.0,0.000000,0.200000,0.600000,...,0.000000,0.000000,0.000000,0.166667,0.0,0.166667,0.20,0.0,0.333333,0.0
#Urban Caf��,0.000000,0.333333,0.0,0.166667,1.000000,0.000000,0.0,0.666667,0.000000,0.166667,...,0.166667,0.000000,0.666667,1.000000,0.0,0.000000,0.00,0.0,0.750000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t Lounge by Dilmah,0.333333,0.000000,0.0,0.166667,0.000000,0.333333,0.0,0.000000,0.250000,0.166667,...,0.000000,0.333333,0.000000,0.000000,0.0,1.000000,0.25,0.0,0.000000,0.0
tashas,0.500000,0.000000,0.0,0.200000,0.000000,0.500000,0.0,0.000000,0.333333,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,1.00,0.0,0.000000,0.0
wagamama,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.200000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,1.0,0.000000,0.0


In [57]:
restaurant = 'Coco Bambu'

sim_score = df_jac.loc[restaurant].sort_values(ascending=False)
sim_score = pd.DataFrame({'Restaurant Name': sim_score.index, 'simScore': sim_score.values})
sim_score = sim_score[(sim_score['Restaurant Name']!= restaurant) & (sim_score['simScore']>=0.6)].head(15)
restaurant_record = pd.merge(sim_score,real_data[['Restaurant Name','Aggregate rating']],how='inner',on='Restaurant Name')
rest_rec = restaurant_record.sort_values('Aggregate rating',ascending=False).drop_duplicates('Restaurant Name',keep='first')

In [58]:
rest_rec

,Restaurant Name,simScore,Aggregate rating
2,Vine - The St. Regis,1.0,4.4
3,Atlas World Grill,1.0,4.3
0,Edison: Food+Drink Lab,1.0,3.9
1,Coral - InterContinental Doha,1.0,3.7
